# 加载LoRA模型

### Step1 导包

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "4"

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

### Step2 加载基础模型

In [2]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh")
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")

### Step3 加载LoRA模型

In [5]:
peft_model = PeftModel.from_pretrained(model, model_id="chatbot/checkpoint-500")
peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): ModulesToSaveWrapper(
          (original_module): Embedding(46145, 2048)
          (modules_to_save): ModuleDict(
            (default): Embedding(46145, 2048)
          )
        )
        (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0): BloomBlock(
            (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
              (dense): Linear(in_features=2048, out_features=2048, bias=True)
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (mlp): BloomMLP(
              (dense_h_to_4h): L

In [9]:
peft_model = peft_model.cuda()
ipt = tokenizer("Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(peft_model.device)
tokenizer.decode(peft_model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True)

'Human: 考试有哪些技巧？\n\nAssistant: 一般来说, 分析题目的目的目的有三个: 一, 考试的意图( 目的是什么? 二, 要达到的目的是什么 ) 三, 答案的范围( 要用什么答案? 给出什么结论 ) 。 例如, 测试题可能有一个特定的意图 ( 学习有关数学的技巧 ), 目标是找到正确答案, 要达到的目的可能会是 " 获得正确分数 " 的结果。 答案的选择可能包括范围的选择, 如: " 将来的选择 "( 或者 " 选择最佳答案 " ), 或可能是一个确定的结论, 如 " 在'

### Step4 模型合并

In [10]:
merge_model = peft_model.merge_and_unload()
merge_model

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(46145, 2048)
    (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
  )
  (l

In [14]:
merge_model = merge_model.cuda()
ipt = tokenizer("Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(merge_model.device)
tokenizer.decode(merge_model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True)

'Human: 考试有哪些技巧？\n\nAssistant: 考试通常有技巧。 如果您打算参加考试，则重要的是要建立良好的学习心态，以在考试中发挥出色。 首先，保持警觉! 没有做错任何事情之前，不要做任何预习。 考生应了解考试内容，以制定复习计划，并根据自己的知识水平，掌握一些可用的技巧，以更好的方式准备考试。\n\n第二，提前准备好你的试卷。 要在考试中发挥出色，考生必须充分利用所有已知的信息。 这可能需要一次或两次测试。 考生还需要保持良好的学习习惯，以便准备考试。'

### Step5 完整模型保存

In [15]:
merge_model.save_pretrained("chatbot/merge_model")